In [12]:

import numpy as np
npa= np.array

epsilon = 1e-10
def LSE(x): 
    x = np.array(x)
    c=np.max(x)
    return c + np.log(np.sum(np.exp(x - c)))

In [14]:
def ql_model(theta,df):

    sideList= df['choice'].tolist()
    rewardList = df['reward'].tolist()
    
    karm= max(sideList)+1
    Qvalue = np.zeros(karm)+(1/karm) 

    alpha = theta[0]
    beta = theta[1]

    probList=[]

    for i in range (len(rewardList)):
        arm = int(sideList[i]) 

        
        ##Softmax function
        prob = np.exp((Qvalue[arm]*beta)-LSE(Qvalue*beta))
        
        probList.append(prob)
        Qvalue [arm] += alpha*(rewardList[i]-Qvalue[arm])

        ##log likelihood
    logLike = (np.sum(np.log(npa(probList)+epsilon))) * (-1)
    
    return logLike

In [15]:
def forage_model(theta,df):  

    
    sideList= df['choice'].tolist()
    rewardList = df['reward'].tolist()
    
   
    oitOreList= []

    #switch/explore = 0, stay/exploit = 1
    for j in range (len(sideList)-1):  
        if sideList[j]==sideList[j+1]:
            oitOreList.append(1)
        else:
            oitOreList.append(0)

    rewardList = rewardList[1:]
    
    

    alpha = theta[0]
    beta = theta[1]   
    rho = theta[2]

    v_oit = 1

    probList=[]
    for i in range (len(rewardList)):
        oit = oitOreList[i] # oit or ore 
            
        if oit == 1:
            prob = np.exp((v_oit*beta)-LSE([v_oit*beta, rho*beta]))
            probList.append(prob)
            v_oit += alpha*(rewardList[i]-v_oit)

        if oit == 0:
            prob = 1-(np.exp((v_oit*beta)-LSE([v_oit*beta, rho*beta])))
            probList.append(prob)
            v_oit = rho + alpha*(rewardList[i]-rho) 
    

    ##log likelihood
    logLike = (np.sum(np.log(npa(probList)+epsilon))) * (-1)
  
    return logLike